<a href="https://colab.research.google.com/github/khailcon/Huehueteotl/blob/Alec_Code/Ceramic_Project_CHEM599.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huehueteotl Ceramic Analysis Project

** https://github.com/khailcon/Huehueteotl **

##Tasks to accomplish
* Edge Detection of ceramics
* Void Detection via inverting color scheme
* Shape Detection
* Automation or For looping for a folder of images
* Share an online folder for raw data that we can access using code
* Make a yml

 

In [45]:
#import all necessary packages
import matplotlib.pyplot as plt
from math import sqrt
import skimage as sk
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
# turn on if using colab
# from google.colab import drive
from pathlib import Path
from skimage.filters import rank
from skimage.morphology import disk, square
from skimage.util import img_as_float, img_as_ubyte
import pandas as pd
import numpy as np


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:

def inclusion_counter(image, blob_algorithm = 'doh', max_sigma = 30, threshold = 0.1):
    """This function returns a rough count of the number of inclusions based on various algorithms

  Parameters
  -----------
  image : image variable
  blob_algorithm : str, optional
    Default: 'doh'. ('dog': Difference of Gaussian, 'log': Laplacian of Gaussian, 'doh': Determinant of Hessian, 'all': Average of all three)
    note- if 'all', threshold for doh will be one magnitude smaller
  max_sigma : float, optional
    Keep this high to detect large blobs
  threshold : float, optional
    Reduce this to detect less prominent blobs

  Returns
  --------
  int
    number of inclusions
  """
    image_gray = rgb2gray(image)
    if blob_algorithm == 'doh':
        blobs = blob_doh(image_gray, max_sigma = max_sigma, threshold = threshold)
        inclusions = len(blobs)
        
    elif blob_algorithm == 'dog':
        blobs = blob_dog(image_gray, max_sigma = max_sigma, threshold = threshold)
        blobs[:,2] = blobs[:, 2] * sqrt(2)
        inclusions = len(blobs)
        
    elif blob_algorithm == 'log':
        blobs = blob_log(image_gray, max_sigma = max_sigma, num_sigma = 10, threshold = threshold)
        blobs[:,2] = blobs[:, 2] * sqrt(2)
        inclusions = len(blobs)
        
    elif blob_algorithm == 'all':
        blobs_doh = blob_doh(image_gray, max_sigma = max_sigma, threshold = .1*threshold)
        blobs_dog = blob_dog(image_gray, max_sigma = max_sigma, threshold = threshold)
        blobs_dog[:,2] = blobs_dog[:, 2] * sqrt(2)
        blobs_log = blob_log(image_gray, max_sigma = max_sigma, num_sigma = 10, threshold = threshold)
        blobs_log[:,2] = blobs_log[:, 2] * sqrt(2)
        inclusions = ((len(blobs_doh)+len(blobs_dog)+len(blobs_log))/3)
    return inclusions

 """This function returns a rough count of the number of inclusions based on various algorithms for a folder of ceramic scans

  Parameters
  -----------
  folder_path : the path of a local folder with images ending in _int.jpeg starting from 1 to n
  blob_algorithm : str, optional
    Default: 'doh'. ('dog': Difference of Gaussian, 'log': Laplacian of Gaussian, 'doh': Determinant of Hessian, 'all': Average of all three)
    note- if 'all', threshold for doh will be one magnitude smaller
  max_sigma : float, optional
    Keep this high to detect large blobs
  threshold : float, optional
    Reduce this to detect less prominent blobs

  Returns
  --------
  DataFrame
    Summary statistics for each ceramic's inclusions
  """

In [75]:

def inclusion_counter2(folder_path, blob_algorithm = 'doh', max_sigma = 30, threshold = 0.1):
    #image selector
    pathlist = Path(folder_path).rglob('*.jpeg')
    images= []
    for path in pathlist:
        image = plt.imread(str(path))
        images.append(image)
        
    
    #image operations
    maxrad = []
    meanrad = []
    minrad = []
    inclusions = []
    names = pathlist #need to get a column in there with names from each ceramic might need to be part of top chunk

    for im in images:
        image_gray = rgb2gray(im)
        names = f'ceramic_{im}'
        if blob_algorithm == 'doh':
        
            blobs = blob_doh(image_gray, max_sigma = max_sigma, threshold = threshold)
            inclusions.append(len(blobs))
        elif blob_algorithm == 'dog':
            blobs = blob_dog(image_gray, max_sigma = max_sigma, threshold = threshold)
            blobs[:,2] = blobs[:, 2] * sqrt(2)
            inclusions.append(len(blobs))

        elif blob_algorithm == 'log':
            blobs = blob_log(image_gray, max_sigma = max_sigma, num_sigma = 10, threshold = threshold)
            blobs[:,2] = blobs[:, 2] * sqrt(2)
            inclusions.append(len(blobs))

        elif blob_algorithm == 'all':
            blobs_doh = blob_doh(image_gray, max_sigma = max_sigma, threshold = .1*threshold)
            blobs_dog = blob_dog(image_gray, max_sigma = max_sigma, threshold = threshold)
            blobs_dog[:,2] = blobs_dog[:, 2] * sqrt(2)
            blobs_log = blob_log(image_gray, max_sigma = max_sigma, num_sigma = 10, threshold = threshold)
            blobs_log[:,2] = blobs_log[:, 2] * sqrt(2)
            inclusions.append((len(blobs_doh)+len(blobs_dog)+len(blobs_log))/3)
    
    
        #maxrad.append(blobs[:,2].max())
        #meanrad.append(blobs[:,2].mean())
        #minrad.append(blobs[:,2].min())
    dataframe = pd.DataFrame(columns =['Names', 'Count', 'Max inclusion size', 'avg inclusion size', 'minimum inclusion size'])
    dataframe['Names'] = names
    dataframe['Count'] = inclusions
    
    return dataframe

In [74]:
folder_path = 'cems2'
pathlist = Path(folder_path).rglob('*.jpeg')
images= []
for path in pathlist:
    image = plt.imread(str(path))
    images.append(image)


(0, array([[[ 69,  69,  71],
        [ 68,  68,  70],
        [ 68,  68,  70],
        ...,
        [ 65,  65,  67],
        [ 65,  65,  65],
        [ 65,  65,  65]],

       [[ 69,  69,  71],
        [ 68,  68,  70],
        [ 68,  68,  70],
        ...,
        [ 65,  65,  67],
        [ 65,  65,  65],
        [ 66,  66,  66]],

       [[ 69,  69,  71],
        [ 69,  69,  71],
        [ 68,  68,  70],
        ...,
        [ 66,  66,  68],
        [ 66,  66,  66],
        [ 66,  66,  66]],

       ...,

       [[ 42,  77,  37],
        [ 42,  77,  37],
        [ 42,  77,  37],
        ...,
        [ 37,  74,  31],
        [ 37,  74,  31],
        [ 36,  73,  30]],

       [[ 42,  79,  36],
        [ 42,  79,  36],
        [ 42,  79,  36],
        ...,
        [ 34,  76,  28],
        [ 34,  76,  28],
        [ 34,  76,  28]],

       [[109,  25, 121],
        [109,  25, 121],
        [109,  25, 121],
        ...,
        [103,  22, 115],
        [103,  22, 115],
        [103,  22, 1

In [58]:
image = plt.imread('cems2/Ceramic_1.jpeg')
image_gray = rgb2gray(image)
blobs = blob_doh(image_gray, max_sigma = 30, threshold = 0.1)
        #blobs[:,2] = blobs[:, 2] * sqrt(2)
        #inclusions.append(len(blobs))\


In [63]:
pd.DataFrame(columns=['one','two'])

,one,two


In [59]:
blobs

array([], shape=(0, 3), dtype=float64)

In [76]:
inclusion_counter2(folder_path, blob_algorithm = 'dog')

,Names,Count,Max inclusion size,avg inclusion size,minimum inclusion size
0,NaN,3390,NaN,NaN,NaN
1,NaN,1230,NaN,NaN,NaN


In [ ]:
impath_google = '/content/drive/MyDrive/Ceramics_Images/Ceramic_13.jpeg'
impath_local = '/Users/aiacobucci922/Documents/CHEM_599/Raw_Images/Ceramic_'
image = plt.imread(impath)
image_gray = rgb2gray(image)

blobs_log = blob_log(image_gray, max_sigma=50, num_sigma=10, threshold=.4)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=.4)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(120, 40), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()

fig.savefig('lowered_sensitivity_large.jpg')

In [ ]:
import pandas as pd
dog_df = pd.DataFrame(data = blobs_dog, columns = ['x location of center', 'y location of center', 'radius (px)'])
dog_df.describe(include='all')

In [ ]:
doh_df = pd.DataFrame(data = blobs_doh, columns = ['x location of center', 'y location of center', 'radius (px)'])
doh_df.describe(include='all')

In [ ]:
from skimage.filters import rank
from skimage.morphology import disk, square
from skimage.util import img_as_float, img_as_ubyte
import pandas as pd
selem = disk(2)



##Run a blur on the image first because these scans seem to be too noisy


In [ ]:
im2 = image
image_gray = rank.mean(rgb2gray(im2), selem = selem)

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=.1)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(30, 10), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(im2)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()

DoH seems to have been over compensated, maybe we should adjust the settings of the blob detector, but DoG and LoG are still finding way too many small boobs where there aren't any. Perhaps adjust counter for a > < of radii? 